## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* metal

In [5]:
start_day = "2012/01/01"

In [65]:
urls = ['https://kr.investing.com/commodities/metals']

In [66]:
url = urls[0]

In [67]:
driver.get(url)
time.sleep(3)

In [68]:
temp_names = driver.find_elements_by_css_selector("table#cross_rate_1 tbody tr td.bold.left.noWrap.elp.plusIconTd a")
temp_flag = driver.find_elements_by_css_selector("table#cross_rate_1 tbody tr td.flag span")

In [69]:
flags = [x.get_attribute("title") for x in temp_flag]

In [70]:
names = [x + "_" + y.text if len(x) != 0 else y.text for x,y in zip(flags, temp_names)]

In [72]:
urls = [x.get_attribute("href") for x in temp_names]

In [77]:
index_url = [x + "-historical-data" if "?" not in x else x.split("?")[0] + "-historical-data?" + x.split("?")[1]for x in urls]
index_names = [x.split("-historical-data")[0].split("/")[-1] for x in index_url]

In [79]:
metal_df = pd.DataFrame({"symbol":index_names,
                         "name":names,
                         "url":index_url})

In [81]:
metal_df.to_csv(data_path + "metal_symbol_df.csv", index = False)

* loop crawling

In [88]:
index_list = []

In [89]:
start_idx = 0

In [90]:
for idx, url in tqdm(enumerate(index_url[start_idx:])):
    driver.get(url)
    time.sleep(5)

    # 날짜 입력란
    tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
    tmp_date.click()
    driver.implicitly_wait(3)

    tmp_input = driver.find_element_by_css_selector("input#startDate")
    time.sleep(0.5)

    # 기존 날짜 지우기
    for _ in range(11) :
        tmp_input.send_keys(Keys.BACKSPACE)
        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)
        
    tmp_input.send_keys(start_day)

    # 요청 버튼
    request_btn = driver.find_element_by_css_selector("a#applyBtn")
    request_btn.click()
    time.sleep(5)
    
    tables = driver.find_elements_by_css_selector("table#curr_table")

    values = tables[0].text.split("\n")[1:]

    values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

    name = names[idx]
    
    index_dict = {"date":[],
                 "{}_close".format(name):[],
                 "{}_vol".format(name):[],
                 "{}_var".format(name):[]}

    for val in values:
        year, month, day, close, _, _, _, volume, variation= val.split(" ")
        date = "-".join([year, month, day])
        index_dict['date'].append(date)
        index_dict['{}_close'.format(name)].append(close)
        index_dict['{}_vol'.format(name)].append(volume)
        index_dict['{}_var'.format(name)].append(variation)
        
    index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
    start_idx += 1

11it [04:16, 23.30s/it]


In [91]:
index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='inner'), index_list)

In [92]:
index_df.tail()

,date,미국_구리_close,미국_구리_vol,미국_구리_var,영국_구리_close,영국_구리_vol,영국_구리_var,금_close,금_vol,금_var,...,영국_알루미늄_var,은_close,은_vol,은_var,영국_주석_close,영국_주석_vol,영국_주석_var,미국_팔라듐_close,미국_팔라듐_vol,미국_팔라듐_var
1924,2014-06-20,3.1180,0.35K,1.40,6818.00,10.21K,1.28,1316.20,0.25K,0.19,...,-0.11,20.942,0.00K,1.48,22525.00,0.94K,0.06,822.95,0.00K,-1.95
1925,2014-06-19,3.0750,0.22K,0.65,6732.00,6.76K,0.22,1313.70,0.60K,3.25,...,0.85,20.637,0.00K,4.40,22512.00,2.07K,0.28,839.35,0.01K,1.91
1926,2014-06-18,3.0550,0.18K,-0.10,6717.00,6.70K,0.03,1272.40,0.14K,0.06,...,1.21,19.767,0.01K,0.23,22450.00,1.66K,-0.35,823.65,0.01K,0.76
1927,2014-06-17,3.0580,0.27K,0.36,6715.00,6.58K,0.48,1271.70,0.82K,-0.25,...,0.64,19.721,8.86K,0.10,22528.00,1.36K,0.04,817.45,0.01K,0.91
1928,2014-06-16,3.0470,0.21K,0.56,6683.00,7.60K,0.62,1274.90,0.33K,0.09,...,0.34,19.702,0.01K,-0.23,22519.00,1.05K,-0.27,810.10,0.00K,-0.40


In [93]:
index_df.to_csv(data_path + "metal_df.csv", index = False)